seperate location terms contain numbers, store them as panda dataframe

In [29]:
!pip install openpyxl

In [ ]:
from os import listdir
from os.path import isfile, join
import re
import requests
import json
import pandas as pd

df = pd.read_csv('deepseek_samples.csv', encoding="UTF-8")
#remove duplication
df['doc_num'] = df['file_number'].str[:-2]
df = df.drop_duplicates(subset=['doc_num', 'location'])
sorted_df =df.sort_values(by=['file_number']).reset_index(drop = True)
sorted_df

In [34]:
numbers_rows = []
location_rows = []
for _, row in sorted_df.iterrows():
     # Regular expression pattern to check if numbers in a string
     pattern = r"\d"
     # Search for the pattern in the text
     if re.search(pattern, row['location']):
        numbers_rows.append({'location': row['location'].strip(), 'doc_num': row['file_number'][:-2]})
     else:
        location_rows.append({'location': row['location'].strip(), 'doc_num': row['file_number'][:-2]})

num_df = pd.DataFrame(numbers_rows)
loc_df = pd.DataFrame(location_rows)

In [ ]:
#load the official LGV address table as reference
address_df = pd.read_csv('hamburg_address.csv')
address_df.head(100)

In [ ]:
# merge LGV house number with street name
address_df['combine'] = address_df['street_name'].str.lower() +  ' ' + address_df['num_zusatz']
address_df

In [ ]:
# small letter geocoded location with house numbers, remove space
num_df['combine'] = num_df['location'].str.lower()
num_df

In [ ]:
import numpy as np
# compare geocoded house number table with LGV data, return coordinates
# common_array = set(num_df['check'].values).intersection(set(address_df['combine']))
common_df = pd.merge(num_df, address_df, on=['combine'], how='inner')
common_df

In [ ]:
compare_df = common_df
compare_df.drop(columns=['stadtteil','cX','cY','num_zusatz','street_name','fid','Thoroughfare'],inplace=True) 
compare_df

In [42]:
no_match = set(num_df['combine']) - set (compare_df['combine'])

no_match_df = pd.DataFrame(columns=num_df.columns)
no_match_df

,location,doc_num,combine


In [ ]:
no_match_df = num_df[num_df['combine'].isin(no_match)].sort_values(by='doc_num').reset_index(drop=True)
no_match_df

In [ ]:
no_match_df['latitude'] = 0
no_match_df['longitude'] = 0
no_match_df

In [ ]:
# call maapbox api for location tables that do not contain house numbers.

from time import sleep

def get_lat_lon(address):
    mapbox_token = "your_mapbox_api_key"  # Replace with your actual Mapbox API key
    url = f"https://api.mapbox.com/search/geocode/v6/forward?q={address}"
    params = {
        'access_token': mapbox_token,
        'limit': 1,
        'language': 'de',  # German language
        'bbox': '9.65067,53.33928,10.39779,53.75625'  # Bounding box for Hamburg area
    }
   
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # Raise an exception for bad status codes
            data = response.json()
            
            if data['features']:
                coordinates = data['features'][0]['geometry']['coordinates']
                print(f"Location: {address}, Coordinates: {coordinates}")
                return address, coordinates[1], coordinates[0]
            else:
                print(f"No results found for address: {address}")
                return address, None, None
        except requests.exceptions.RequestException as e:
            print(f"Error: {e} for address: {address}")
            if attempt < max_retries - 1:
                print(f"Retrying... (Attempt {attempt + 2}/{max_retries})")
                sleep(1)  # Wait for 1 second before retrying
            else:
                print(f"Max retries reached for address: {address}")
                return address, None, None

def geocode_dataframe(no_match_df, location_column='location'):
    geocoded_data = []
    for address in no_match_df[location_column]:
        location, lat, lon = get_lat_lon(address)
        geocoded_data.append({'location': location, 'latitude': lat, 'longitude': lon})
        sleep(0.1)  # Add a small delay to avoid hitting rate limits
    
    geocoded_df = pd.DataFrame(geocoded_data)
    return pd.concat([no_match_df, geocoded_df[['latitude', 'longitude']]], axis=1)

# Usage example
geocoded_df = geocode_dataframe(no_match_df)
print(geocoded_df)

In [56]:
geocoded_df

In [ ]:
common_df

In [51]:
import openpyxl
common_df.to_excel('geocode_deepseek_lgv.xlsx', index=False)
geocoded_df.to_excel('geocode_deepseek_mapbox.xlsx', index=False)

In [ ]:
#geocoded addresses without house numbers with mapbox
geocoded_df = geocode_dataframe(loc_df)
print(geocoded_df)

In [ ]:
geocoded_df.to_excel('geocode_deepseek_no_numbers.xlsx', index=False)